Librerias

In [5]:
from afip_data_transformation import transform_afip_outbound_invoices # -> Transforma los comprobantes en formato que necesitamos
from bookit_data_transformation import procesar_outbound_html # -> Transforma el registro de afip
from data_comparison import comparar_facturas_venta # -> Compara ambos registros y devuelve una lista de los que no están actualizados
import os
import lxml
import pandas as pd
import string
import numpy as np

afip_data = pd.read_excel('../downloads/comprobantes_emitidos.xlsx', header=0, skiprows=1)

afip_data_outbound = transform_afip_outbound_invoices(afip_data)
bookit_data_outbound = procesar_outbound_html('../downloads/outbound.html')
clean_data = comparar_facturas_venta(afip_data_outbound,bookit_data_outbound,1)

✅ Archivo ../downloads/outbound.html procesado correctamente (50 filas).
⚠️  6 facturas no están cargadas en el sistema.
✅ 6 facturas correctamente cargadas y listas para procesar.


c:\Users\ThinkPad-PC\Documents\GitHub\AFIP-nuevo\src\data_comparison.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  merged.loc[~merged['loaded'], 'totales_ok'] = np.nan
c:\Users\ThinkPad-PC\Documents\GitHub\AFIP-nuevo\src\data_comparison.py:57: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mal_cargadas = merged[(merged['loaded']) & (~merged['totales_ok'].fillna(False))]


In [2]:
clean_data

,Fecha,tipo2_new,tipo3_new,Factura,Cliente,CUIT,TOTAL_10.5,TOTAL_21,TOTAL_NO_GRAVADO
48,17/11/2025,Factura,A,3326,CRETA SRL,33710864409,0.0,125514.16,0.0
49,17/11/2025,Factura,A,3327,CRETA SRL,33710864409,0.0,125514.16,0.0


In [6]:
#LIBRERIAS DEL PROYECTO

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

#Este es driver que va a navegar.
driver = webdriver.Chrome()

# Función para iniciar sesión
arancia_url = 'https://arancia-bookit.com/'

driver.get(arancia_url)
time.sleep(3)

# Colocar las claves
element = driver.find_element(By.ID, 'TextBox1')
element.send_keys('FRANCESC')
element = driver.find_element(By.ID, 'TextBox2')
element.send_keys('n4r4nj4720')

# click en entrar
element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

# clickear en la pestaña de "facturación"
element = driver.find_element(By.ID,"Button10")
element.click()
time.sleep(3)

# clickear en la sub-pestaña de "facturación" (para cargar facturas de ventas)
driver.switch_to.frame('facturacion')
element = driver.find_element(By.ID,"Button5")
element.click()
time.sleep(1)

driver.switch_to.frame('marco')

# Quitar check de "aéreo"
element = driver.find_element(By.ID,"CheckBoxList1_0")
element.click()
time.sleep(2)

# Poner check en "manual"
element = driver.find_element(By.ID,"CheckBoxList1_3")
element.click()
time.sleep(2)

# Buscar el input de IVAS
element = driver.find_element(By.ID, "contar")

# Seleccionar todo el texto actual
element.send_keys(Keys.CONTROL, "a")
time.sleep(0.2)

# Escribir el nuevo valor (3 por la información que manejamos)
element.send_keys("3")

# Disparar el evento de cambio usando TAB (como si el usuario saliera del campo)
element.send_keys(Keys.TAB)
time.sleep(3)

# Valores específicos para cada campo
valores = {
    "ivasa1": "0",
    "ivasa2": "10.5"  # con punto, ojo a la compa que rompe la página
}

for field_id, valor in valores.items():
    element = driver.find_element(By.ID, field_id)

    # Escribir directamente el nuevo valor (sin dejar el campo vacío)
    element.send_keys(Keys.CONTROL, 'a')
    element.send_keys(valor)
    element.send_keys(Keys.TAB)
    time.sleep(3)

for index, row in clean_data.iterrows():
    # TOTAL_NO_GRAVADO
    elem = driver.find_element(By.ID, 'total1')
    elem.send_keys(Keys.CONTROL, 'a')  # Selecciona todo
    elem.send_keys(row['TOTAL_NO_GRAVADO'])  # Sobreescribe
    elem.send_keys(Keys.TAB)
    time.sleep(3)

    # TOTAL_10.5
    elem = driver.find_element(By.ID, 'total2')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['TOTAL_10.5'])
    elem.send_keys(Keys.TAB)
    time.sleep(3)

    # TOTAL_21
    elem = driver.find_element(By.NAME, 'total3')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['TOTAL_21'])
    elem.send_keys(Keys.TAB)
    time.sleep(3)

    driver.find_element(By.ID, "Button2").click()
    time.sleep(3)

    # RAZON SOCIAL
    elem = driver.find_element(By.NAME, 'TextBox3')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['Cliente'])
    elem.send_keys(Keys.TAB)
    time.sleep(1)

    # CUIT
    elem = driver.find_element(By.NAME, 'TextBox4')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['CUIT'])
    elem.send_keys(Keys.TAB)
    time.sleep(1)   

    # Factura NÚMERO
    elem = driver.find_element(By.NAME, 'TextBox6')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['Factura'])
    elem.send_keys(Keys.TAB)
    time.sleep(1)  

    # Clase
    elem = driver.find_element(By.NAME, 'TextBox7')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['tipo3_new'])
    elem.send_keys(Keys.TAB)
    elem = driver.find_element(By.NAME, 'TextBox8')   
    elem.send_keys(row['Fecha'])
    time.sleep(1)

    # Click en Grabar
    driver.find_element(By.ID, "Button5").click()
    time.sleep(2)


Iniciar sesión

In [4]:
# Función para iniciar sesión
arancia_url = 'http://arancia-001-site1.btempurl.com/'

driver.get(arancia_url)
time.sleep(3)

# Entrar a la Web
element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

# Colocar las claves
element = driver.find_element(By.ID, 'TextBox1')
element.send_keys('FRANCESC')
element = driver.find_element(By.ID, 'TextBox2')
element.send_keys('n4r4nj4720')

# click en entrar
element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

# clickear en la pestaña de "facturación"
element = driver.find_element(By.ID,"Button10")
element.click()
time.sleep(3)

# clickear en la sub-pestaña de "facturación" (para cargar facturas de ventas)
driver.switch_to.frame('facturacion')
element = driver.find_element(By.ID,"Button5")
element.click()
time.sleep(1)

Cambiar de frame al marco interno

In [5]:
driver.switch_to.frame('marco')

Establecer factura manual

In [6]:
# Quitar check de "aéreo"
element = driver.find_element(By.ID,"CheckBoxList1_0")
element.click()
time.sleep(2)

# Poner check en "manual"
element = driver.find_element(By.ID,"CheckBoxList1_3")
element.click()
time.sleep(2)

# Buscar el input de IVAS
element = driver.find_element(By.ID, "contar")

# Seleccionar todo el texto actual
element.send_keys(Keys.CONTROL, "a")
time.sleep(0.2)

# Escribir el nuevo valor (3 por la información que manejamos)
element.send_keys("3")

# Disparar el evento de cambio usando TAB (como si el usuario saliera del campo)
element.send_keys(Keys.TAB)
time.sleep(3)

CAMBIAR LOS IVAS

In [7]:
# Valores específicos para cada campo
valores = {
    "ivasa1": "0",
    "ivasa2": "10.5"  # con punto, ojo a la compa que rompe la página
}

for field_id, valor in valores.items():
    element = driver.find_element(By.ID, field_id)

    # Escribir directamente el nuevo valor (sin dejar el campo vacío)
    element.send_keys(Keys.CONTROL, 'a')
    element.send_keys(valor)
    element.send_keys(Keys.TAB)
    time.sleep(3)

Procesar las facturas para cargar datos

In [4]:
import pandas as pd
import string
import numpy as np
import re

data = pd.read_excel(r'../downloads/comprobantes_emitidos.xlsx', header=0, skiprows=1)

#TIPO DE FACTURA
data['codigo_fc'] = [int(re.match(r'(\d+)', x).group(1)) for x in data['Tipo']]
data['Tipo'] = data['Tipo'].astype('string').str[-9:]
data['Tipo2'] = data['Tipo'].str[:7]
data['Tipo3'] = data['Tipo'].str[-1:]

data['tipo2_new'] = np.where(data['codigo_fc'].isin([1,6,11]), "Factura", "Credito")
data['tipo2_new'] = np.where(data['codigo_fc'] == 201, "Pyme_fc", data['tipo2_new'])
data['tipo2_new'] = np.where(data['codigo_fc'] == 203, "Pyme_nc", data['tipo2_new'])

data['tipo3_new'] = np.where(data['codigo_fc'].isin([1,3,201,203]), "A", "C")
data['tipo3_new'] = np.where(data['codigo_fc'].isin([6,8]), "B", data['tipo3_new'])

data['tipo3_new']

# FACTURA
data['Punto de Venta'] = data['Punto de Venta'].astype(str).str.zfill(5)
data['Número Desde'] = data['Número Desde'].astype(str).str.zfill(8)
data['Factura'] = data['Número Desde'].astype(int)

# Cliente
data['Denominación Receptor'] = data['Denominación Receptor'].str.translate(
    str.maketrans('', '', string.punctuation))
data['Cliente'] = data['Denominación Receptor'].str[:35]

# CUIT
data['CUIT'] = data['Nro. Doc. Receptor'].astype(str)


# IMPORTES
data['NETO 10.5'] = data['Neto Grav. IVA 10,5%'] 
data['IVA 10.5'] = data['IVA 10,5%'] 
data['NETO 21'] = data['Neto Grav. IVA 21%'] 
data['IVA 21'] = data['IVA 21%'] 
data['NO GRAVADO'] = data['Neto No Gravado'] 
data['EXENTO'] = data['Op. Exentas'] 
data['IMPUESTOS'] = data['Otros Tributos']
data['NETO 0'] = data['Neto Grav. IVA 0%']
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])
data['TOTAL_10.5'] = round(data['NETO 10.5'] + data['IVA 10.5'],2)
data['TOTAL_21'] = round(data['NETO 21'] + data['IVA 21'],2)


# PONER 0 A TODAS LAS COLUMNAS QUE NO TIENEN ALGO
columnas = ['NETO 0', 'NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS',
            'IVA 2,5%', 'IVA 5%', 'IVA 27%', 'TOTAL_NO_GRAVADO', 'TOTAL_10.5', 'TOTAL_21'] 
for col in columnas: 
    data[col] = data[col].fillna(0)

# TIPO DE CAMBIO ESTE CAMBIAR PORQUE CAMBIA A FUTURO
columnas_tc = ['NETO 0','NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS', 'TOTAL_10.5', 'TOTAL_21'] 
for col in columnas_tc: 
    data[col] = data[col] * data['Tipo Cambio']

# SUMAR LO QUE NO LLEVA IVA
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])

# Convertimos las columnas a str
data['NETO 10.5'] = data['NETO 10.5'].astype(str)
data['IVA 10.5'] = data['IVA 10.5'].astype(str)
data['NETO 21'] = data['NETO 21'].astype(str)
data['IVA 21'] = data['IVA 21'].astype(str)
data['TOTAL_21'] = data['TOTAL_21'].astype(str)
data['TOTAL_10.5'] = data['TOTAL_10.5'].astype(str)

# PONER EN NEGATIVO LAS COLUMNAS QUE SON CREDITO
for col in columnas:
    mask = (data['tipo2_new'] == 'Credito') | (data['tipo2_new'] == 'Pyme_nc')
    data.loc[mask, col] = -data.loc[mask, col].astype(float)
    data[col] = data[col].astype(str)

# CREAR Y APLICAR MÁSCARA
clean = ['Fecha','tipo2_new','tipo3_new', 'Factura',
        'Cliente','CUIT', 'TOTAL_10.5', 'TOTAL_21',
        'TOTAL_NO_GRAVADO']

clean_data = data[clean]
clean_data

,Fecha,tipo2_new,tipo3_new,Factura,Cliente,CUIT,TOTAL_10.5,TOTAL_21,TOTAL_NO_GRAVADO
0,03/11/2025,Factura,A,3301,NEW PGA S R L,30615265450,0.0,46892.22,292182.75
1,03/11/2025,Factura,A,3302,NEW PGA S R L,30615265450,698402.31,1077111.06,14317.69
2,03/11/2025,Factura,A,3303,NEW PGA S R L,30615265450,1057484.68,420649.47,28635.32
3,03/11/2025,Factura,B,3619,LA IGLESIA UNIVERSAL DEL REINO DE D,33648976289,0.0,0.0,2178675.2
4,03/11/2025,Factura,B,3620,UNION PERSONAL DE SEGURIDAD DE LA R,30552442608,0.0,0.0,1056927.17
5,03/11/2025,Factura,B,3621,LA IGLESIA UNIVERSAL DEL REINO DE D,33648976289,0.0,0.0,227637.76
6,03/11/2025,Factura,B,3617,LA IGLESIA UNIVERSAL DEL REINO DE D,33648976289,0.0,0.0,2585369.0
7,03/11/2025,Factura,B,3618,LA IGLESIA UNIVERSAL DEL REINO DE D,33648976289,0.0,0.0,990336.0
8,04/11/2025,Factura,A,3304,TASA LOGISTICA SA,30629145458,1189709.3,130914.74,28635.42
9,05/11/2025,Factura,A,3305,NEWSAN SOCIEDAD ANONIMA,30642617555,1065364.91,87839.72,42953.12


In [ ]:
from afip_data_transformation import transform_afip_outbound_invoices # -> Transforma los comprobantes en formato que necesitamos
from bookit_data_transformation import procesar_outbound_html # -> Transforma el registro de afip
from data_comparison import comparar_facturas_venta # -> Compara ambos registros y devuelve una lista de los que no están actualizados
import os
import lxml
import pandas as pd
import string
import numpy as np

afip_data = pd.read_excel('../downloads/comprobantes_emitidos.xlsx', header=0, skiprows=1)

afip_data_outbound = transform_afip_outbound_invoices(afip_data)
bookit_data_outbound = procesar_outbound_html('../downloads/outbound.html')
clean_data = comparar_facturas_venta(afip_data_outbound,bookit_data_outbound,1)

Hemos logrado crear un dataset con todas las facturas que tiene el sistema, ahora solo falta que las cargue.


De aquí hacia abajo no funciona la cosa

----------------------



In [9]:
for index, row in clean_data.iterrows():
    # TOTAL_NO_GRAVADO
    elem = driver.find_element(By.ID, 'total1')
    elem.send_keys(Keys.CONTROL, 'a')  # Selecciona todo
    elem.send_keys(row['TOTAL_NO_GRAVADO'])  # Sobreescribe
    elem.send_keys(Keys.TAB)
    time.sleep(3)

    # TOTAL_10.5
    elem = driver.find_element(By.ID, 'total2')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['TOTAL_10.5'])
    elem.send_keys(Keys.TAB)
    time.sleep(3)

    # TOTAL_21
    elem = driver.find_element(By.NAME, 'total3')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['TOTAL_21'])
    elem.send_keys(Keys.TAB)
    time.sleep(3)

    driver.find_element(By.ID, "Button2").click()
    time.sleep(3)

    # RAZON SOCIAL
    elem = driver.find_element(By.NAME, 'TextBox3')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['Cliente'])
    elem.send_keys(Keys.TAB)
    time.sleep(1)

    # CUIT
    elem = driver.find_element(By.NAME, 'TextBox4')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['CUIT'])
    elem.send_keys(Keys.TAB)
    time.sleep(1)   

    # Factura NÚMERO
    elem = driver.find_element(By.NAME, 'TextBox6')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['Factura'])
    elem.send_keys(Keys.TAB)
    time.sleep(1)  

    # Clase
    elem = driver.find_element(By.NAME, 'TextBox7')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['tipo3_new'])
    elem.send_keys(Keys.TAB)
    elem = driver.find_element(By.NAME, 'TextBox8')   
    elem.send_keys(row['Fecha'])
    time.sleep(1)

    # Click en Grabar
    driver.find_element(By.ID, "Button5").click()
    time.sleep(2)


In [43]:
# os.remove(r'C:\Users\ThinkPad-PC\Downloads\Mis Comprobantes Recibidos - CUIT 30714894346.xlsx')